In [1]:
from neo4j import GraphDatabase

user_id = 1110008753489760261
tweet_id = 1109874238398320640
politic_usa_id = 14792049

uri = 'bolt://localhost:7687'
driver = GraphDatabase.driver(uri)

In [2]:
def retrieve_all_tweet_stories():
    message = """MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {type: 'TWEET'})
                 WHERE NOT (start)-[:REPLIED_TO|RETWEETED_FROM]->() AND NOT ()-[:REPLIED_TO|RETWEETED_FROM]->(end)
                 WITH *, relationships(p) AS r
                 RETURN end.created_at, start.created_at, r"""
    records = driver.session().run(message).data()
    return records

In [3]:
def retrieve_tweet_story(tweet_id):
    message = """MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {type: 'TWEET'})
                 WHERE NOT (start)-[:REPLIED_TO|RETWEETED_FROM]->() AND NOT ()-[:REPLIED_TO|RETWEETED_FROM]->(end)
                 AND start.id = {id}
                 WITH *, relationships(p) AS r
                 RETURN end.created_at, start.created_at, r"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [4]:
def retrieve_users_who_interacted_at_a_story(tweet_id):
    message = """MATCH u=(user:User)-[:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet)<-[:TWEETED]-(storyStarter:User)
                 WHERE start.id = {id} AND (end.type = 'RETWEET' OR end.type = 'REPLY') AND start.type = 'TWEET'
                 RETURN user, storyStarter"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [5]:
def retrieve_stories_where_user_is_involved(user_id):
    message = """MATCH u=(user:User)-[:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet)<-[:TWEETED]-(storyStarter:User)
                 WHERE user.id = {id} or storyStarter.id = {id} AND (end.type = 'RETWEET' OR end.type = 'REPLY') AND start.type = 'TWEET'
                 WITH *, relationships(u) as re
                 RETURN end, re, start, user, storyStarter"""
    records = driver.session().run(message, id=user_id).data()
    return records

In [6]:
def retrieve_tweet_chain_ordered_by_time(tweet_id):
    # not needed, tweet chains are created in order in the first place
    print()

In [7]:
def retrieve_stories_made_by_user(user_id):
    message = """MATCH u=(user:User)-[:TWEETED]->(tweet:Tweet)
                 WHERE user.id = {id}
                 MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(tweet:Tweet)
                 WHERE end.type = 'RETWEET' OR end.type = 'REPLY' AND tweet.type = 'TWEET'
                 RETURN u"""
    records = driver.session().run(message, id=user_id).value()
    return records

In [8]:
def retrieve_time_spread_of_tweet_story(tweet_id):
    records = retrieve_tweet_story(tweet_id)[0]
    start_time = records['start.created_at']
    end_time = records['end.created_at']
    message = "Original tweet was posted on %s. \nLast available reaction was posted on %s." % (start_time, end_time)
    
    return message

In [9]:
def trial_and_error(tweet_id):
    message = """MATCH u=(user:User)-[r:POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: {id}, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
                 WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
                 RETURN user, end.created_at as timestamp"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [10]:
# print(retrieve_tweet_story(1109874238398320640))
# print(retrieve_users_who_interacted_at_a_story(1109874238398320640))
# for value in retrieve_stories_made_by_user(politic_usa_id):
#     print(value)
#     print("HELLO DARKNESS WORLD")
#     print("\n")

# print(retrieve_time_spread_of_tweet_story(tweet_id))

# print(retrieve_users_who_interacted_at_a_story(tweet_id))
print(trial_and_error(tweet_id))


[{'user': <Node id=77671 labels={'User'} properties={'followers': 354, 'id': 1109874352479047680, 'screen_name': 'erinbaby', 'friends': 457}>, 'timestamp': 'Sun Mar 24 17:47:20 EET 2019'}, {'user': <Node id=77673 labels={'User'} properties={'followers': 2880, 'id': 1109874372754460675, 'screen_name': 'MiamiNice57', 'friends': 2787}>, 'timestamp': 'Sun Mar 24 17:47:25 EET 2019'}, {'user': <Node id=77675 labels={'User'} properties={'followers': 136, 'id': 1109874595203407874, 'screen_name': 'silverthornn264', 'friends': 216}>, 'timestamp': 'Sun Mar 24 17:48:18 EET 2019'}, {'user': <Node id=77677 labels={'User'} properties={'followers': 2863, 'id': 1109874851576188928, 'screen_name': 'MarianMme16513', 'friends': 4861}>, 'timestamp': 'Sun Mar 24 17:49:19 EET 2019'}, {'user': <Node id=77679 labels={'User'} properties={'followers': 173, 'id': 863156070482534400, 'screen_name': 'WhobbaBobba', 'friends': 86}>, 'timestamp': 'Sun Mar 24 17:49:21 EET 2019'}, {'user': <Node id=77681 labels={'User'

In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# WITH collect(user) as users, storyStarter, collect(end) as tweets, r, start
# UNWIND users AS user
# UNWIND tweets AS tweet
# MATCH pa=(someone:User)-[:POSTED_REPLY|POSTED_RETWEET]->(post:Tweet)-[:REPLIED_TO|RETWEETED_FROM]->(tweet)<-[:POSTED_REPLY|POSTED_RETWEET]-(user)
# WHERE post.created_at < tweet.created_at AND (exists((user)-[:FOLLOWS]->(someone)) OR exists((user)-[:FOLLOWS]->(storyStarter)))
# RETURN someone, user


In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# RETURN u


In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# WITH collect(user) as users, storyStarter, collect(end) as tweets, r
# RETURN users, storyStarter

In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# WITH collect(user) as users, storyStarter, collect(end) as tweets, r, start
# RETURN tweets, start